In [1]:
import sys
sys.path.append("../src/")

In [2]:
%load_ext autoreload
%autoreload 2

import pydevil

/home/salvatore.milite/miniconda3/envs/scdeepaa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Using SVI in combination with HMC

We will use the dataset published in 

In [3]:
import scanpy as sc
import patsy as ptsy
import torch
import numpy as np

In [4]:
adata = sc.datasets.pbmc3k()
adata_proc = sc.datasets.pbmc3k_processed()
adata = adata[adata_proc.obs_names, :]
adata.obs = adata_proc.obs

In [5]:
library_size_factors = adata.obs["n_counts"] / adata.obs["n_counts"].mean()
covariates = ptsy.dmatrix("~ louvain - 1", adata.obs)

In [6]:
res_de  = pydevil.run_SVDE(adata.X.todense(), 
                        covariates, # new design matrix 
                        library_size_factors, 
                        gene_names = adata.var_names,
                        cell_names = adata.obs_names, 
                        optimizer_name = "Adam",
                        lr = 0.05,
                        gamma_lr = 0.01,
                        steps = 200, 
                        full_cov = True, 
                        cuda = True)

ELBO: -10964443386597886394368.00000  : 100%|██████████| 200/200 [01:20<00:00,  2.48it/s]


In [7]:
contrast = np.array([1,-1,0,0,0,0,0,0]) # CD4T vs B cells

In [16]:
marker_genes = pydevil.test_posterior_null(res_de, contrast, 0.05)

In [9]:
marker_genes_name = marker_genes[marker_genes["is_significant"]]

In [10]:
marker_genes_name

,gene,log_FC,p_value,p_value_adj,is_significant
5,AL627309.1,-0.494089,9.261503e-04,1.119244e-02,True
35,NOC2L,-0.624918,1.383601e-04,2.161084e-03,True
36,KLHL17,-0.427786,1.237445e-03,1.428472e-02,True
37,PLEKHN1,-0.391334,6.070079e-04,7.848051e-03,True
40,HES4,-1.569467,0.000000e+00,0.000000e+00,True
...,...,...,...,...,...
32700,MT-ATP8,-0.760710,2.018041e-03,2.134624e-02,True
32701,MT-ATP6,0.194571,4.458177e-03,4.094970e-02,True
32703,MT-ND3,-0.954462,1.099121e-14,7.480877e-13,True
32711,AC145212.1,-0.559175,6.623314e-05,1.115976e-03,True


In [11]:
marker_genes_name = marker_genes_name[np.abs(marker_genes_name["log_FC"]) > 1]["gene"]

In [12]:
marker_genes_name

40          HES4
70        MRPL20
93          NADK
189       CAMTA1
196        VAMP3
          ...   
32543        MX1
32576     NDUFV3
32594       PDXK
32598     AGPAT3
32649    FAM207A
Name: gene, Length: 1029, dtype: object

In [13]:
res_de_HMC  = pydevil.run_HMC(adata[:,marker_genes_name].X.todense(), 
                        covariates, # new design matrix 
                        library_size_factors, 
                        gene_names = marker_genes_name,
                        cell_names = adata.obs_names, 
                        num_samples = 100,
                        num_chains = 2,
                        warmup_steps = 100,
                        full_cov = True, 
                        cuda = True)

/home/salvatore.milite/work/python_packages/pydevil/docs/../src/pydevil/interface.py:219: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_matrix, model_matrix, UMI = torch.tensor(input_matrix).float(), torch.tensor(model_matrix).float(), torch.tensor(ncounts).float()
Warmup [1]:  42%|████▏     | 84/200 [08:01,  6.40s/it, step size=3.93e-06, acc. prob=0.723]


Sample [1]:  66%|██████▌   | 132/200 [15:36, 11.16s/it, step size=1.91e-03, acc. prob=0.719]


Sample [2]: 100%|██████████| 200/200 [23:20,  7.00s/it, step size=1.31e-01, acc. prob=0.707]


In [20]:
pydevil.test_posterior_ROPE_HMC(res_de_HMC, contrast)

,gene,log_FC,ROPE
40,HES4,-0.000586,1.000
70,MRPL20,-0.019467,1.000
93,NADK,0.013631,0.995
189,CAMTA1,-0.000525,1.000
196,VAMP3,-0.053875,1.000
...,...,...,...
32543,MX1,0.001852,1.000
32576,NDUFV3,0.001428,0.990
32594,PDXK,0.013227,1.000
32598,AGPAT3,0.031131,1.000
